In [ ]:
import pandas as pd
import requests
from requests.exceptions import ChunkedEncodingError
import os
from pathlib import Path

In [ ]:
def download_dataset(columns, database_id, filepath, arguments="", 
                     url="https://dados.uffs.edu.br/api/3/action/datastore_search_sql?sql="):
    offset = 0
    limit = 32001
    dataset = []
    while True:
        sql_query = f"SELECT {columns} FROM \"{database_id}\" {arguments} LIMIT {limit} OFFSET {offset}"
        new_url = f"{url}{sql_query}"
        try:
            # Make the GET request to fetch the data
            response = requests.get(new_url, verify=False)
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                # Check if data is found
                if data['success']:
                    records = data['result']['records']
                    # If no more records are returned, exit the loop
                    if not records:
                        break
                    # Append the current chunk of records to the list
                    dataset.extend(records)
                    # Increment the offset for the next chunk
                    offset += limit
                else:
                    print("Nenhum dado encontrado")
                    break
            else:
                print(f"Erro na solicitação: Código de status {response.status_code}")
                break
        except ChunkedEncodingError:
            print("Erro de conexão. Tentando novamente...")
            continue
        # Create a DataFrame from all the retrieved records
    dataset_df = pd.DataFrame(dataset)
    os.makedirs(Path(filepath).parent, exist_ok=True)

    # Save the DataFrame to a CSV file
    dataset_df.to_csv(filepath, index=False)

# CCRS DE CHAPECÓ

In [ ]:
columns = "cod_ccr, nome_ccr, unid_responsavel, cod_cursos, nome_cursos"

database_id = "c51852e5-c5ae-4ce2-97d8-9261033b8877"
filepath = "../../src/input/ccrs_uffs.csv"

arguments = "WHERE unid_responsavel = 'COORDENAÇÃO ACADÊMICA - CHAPECÓ' and LOWER(nome_cursos) LIKE '%computação%'"
# download_dataset("*", database_id, arguments, "test.csv")
download_dataset(columns, database_id, filepath, arguments)

# Turmas Graduação

In [ ]:
columns = "*"

database_id = "670bc744-2570-4e2a-b73b-3acf2039a7c1"
filepath = "../../src/input/turmas_graduacao.csv"

arguments = "WHERE nome_campus = 'Chapecó' AND ano >= 2018"
# AND (cursos_turma = 'CIÊNCIA DA COMPUTAÇÃO' OR cursos_turma = 'ENGENHARIA AMBIENTAL E SANITÁRIA' OR cursos_turma = 'MATEMÁTICA')

download_dataset(columns, database_id, filepath, arguments)

# Histórico Graduação

In [ ]:
columns = "*"

database_id = "3c31f55b-be46-4ddc-9cd5-f10f614f06bc"
filepath = "../../src/input/historico_graduacao.csv"

arguments = "WHERE ano >= 2018"
# nome_campus = 'Chapecó' AND (nome_curso = 'CIÊNCIA DA COMPUTAÇÃO' OR nome_curso = 'ENGENHARIA AMBIENTAL E SANITÁRIA' OR nome_curso = 'MATEMÁTICA') AND

download_dataset(columns, database_id, filepath, arguments)
